# Notebook to conduct model development

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from scipy.stats import skew  
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold as kfolds
from sklearn.metrics import mean_squared_log_error
from math import sqrt
from sklearn import model_selection 
import matplotlib.pylab as plt 
from matplotlib.pylab import rcParams


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

from google.colab import drive
drive.mount('/content/drive')
print("Imported Libraries")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Sat Apr 11 16:27:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/W207")

### Attempting GPU Training

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/W207/xgboost")

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/W207/xgboost/python-package")

In [0]:
!ls

MANIFEST.in  README.rst  setup.cfg  setup.py  xgboost


In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/W207/xgboost/build")

In [0]:
!cmake .. -DUSE_CUDA=ON -DUSE_NCCL=ON -DNCCL_ROOT=/path/to/nccl2

-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- CMake version 3.12.0
-- Performing Test XGBOOST_MM_PREFETCH_PRESENT
-- Performing Test XGBOOST_MM_PREFETCH_PRESENT - Success
-- Performing Test XGBOOST_BUILTIN_PREFETCH_PRESENT
-- Performing Test XGBOOST_BUILTIN_PREFETCH_PRESENT - Success
-- xgboost VERSION: 1.1.0
-- Setting build type to 'Release' as none was specified.
-- Configured CUDA host compiler: /usr/bin/c++
-- The CUDA compiler identificati

In [0]:
!make -j4

Scanning dependencies of target rabit_base
Scanning dependencies of target dmlc
Scanning dependencies of target rabit
Scanning dependencies of target objxgboost
[ -2%] Building CXX object rabit/CMakeFiles/rabit_base.dir/src/allreduce_base.cc.o
[ -1%] Building CXX object rabit/CMakeFiles/rabit.dir/src/allreduce_base.cc.o
[ -1%] Building CXX object dmlc-core/CMakeFiles/dmlc.dir/src/config.cc.o
[  0%] Building CXX object dmlc-core/CMakeFiles/dmlc.dir/src/data.cc.o
[  1%] Building CXX object rabit/CMakeFiles/rabit.dir/src/allreduce_robust.cc.o
[  2%] Building CXX object rabit/CMakeFiles/rabit_base.dir/src/engine_base.cc.o
[  3%] Building CXX object rabit/CMakeFiles/rabit_base.dir/src/c_api.cc.o
[  4%] Building CXX object src/CMakeFiles/objxgboost.dir/c_api/c_api.cc.o
[  5%] Linking CXX static library librabit_base.a
[  5%] Built target rabit_base
Scanning dependencies of target rabit_empty
[  6%] Building CXX object rabit/CMakeFiles/rabit_empty.dir/src/engine_empty.cc.o
[  7%] Building CXX

In [0]:
!python setup.py install --use-cuda --use-nccl

/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:472: UserWarning: The version specified ('1.1.0-SNAPSHOT') is an invalid version, this may not work as expected with newer versions of setuptools, pip, and PyPI. Please see PEP 440 for more details.
  "details." % version
running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing xgboost.egg-info/PKG-INFO
writing dependency_links to xgboost.egg-info/dependency_links.txt
writing requirements to xgboost.egg-info/requires.txt
writing top-level names to xgboost.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'xgboost.egg-info/SOURCES.txt'
running build_ext
creating build/temp.linux-x86_64-3.6/xgboost/src
copying /content/drive/My Drive/Colab Notebooks/W207/xgboost/python-package/../src/CMakeLists.txt -> build/temp.linux

### Read in cleaned data

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/W207")
df = pd.read_csv('GPU.csv')
y = df['SalePrice']
df = df.drop(['SalePrice'], axis = 1)
dfTest = pd.read_csv('newTest.csv')
dfTest = dfTest.set_index('Unnamed: 0')
dfTest.index.names = ['']

### To split data

In [0]:
X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

### Get condensed Dataset from stepwise

In [0]:
condensedTest = X_val[['LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
                              'GrLivArea', 'Fireplaces', 'ScreenPorch', 'MSSubClass_160', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RL', 'LandSlope_Gtl', 'LandSlope_Mod', 'Neighborhood_BrkSide', 'Neighborhood_Crawfor', 
'Neighborhood_Edwards', 'Neighborhood_StoneBr', 'Condition1_Artery', 'Condition1_RRAe', 'OverallCond_3', 'OverallCond_4', 'OverallCond_5', 'OverallCond_6', 'BsmtExposure_Gd', 'Heating_Grav', 'KitchenQual_Ex', 'Functional_Maj2', 
      'Functional_Mod', 'GarageType_2Types', 'GarageQual_Ex', 'SaleCondition_Normal']]

condensedData = X_train[['LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
                              'GrLivArea', 'Fireplaces', 'ScreenPorch', 'MSSubClass_160', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RL', 'LandSlope_Gtl', 'LandSlope_Mod', 'Neighborhood_BrkSide', 'Neighborhood_Crawfor', 
'Neighborhood_Edwards', 'Neighborhood_StoneBr', 'Condition1_Artery', 'Condition1_RRAe', 'OverallCond_3', 'OverallCond_4', 'OverallCond_5', 'OverallCond_6', 'BsmtExposure_Gd', 'Heating_Grav', 'KitchenQual_Ex', 'Functional_Maj2', 
      'Functional_Mod', 'GarageType_2Types', 'GarageQual_Ex', 'SaleCondition_Normal']]

finalTest = dfTest[['LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
                              'GrLivArea', 'Fireplaces', 'ScreenPorch', 'MSSubClass_160', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RL', 'LandSlope_Gtl', 'LandSlope_Mod', 'Neighborhood_BrkSide', 'Neighborhood_Crawfor', 
'Neighborhood_Edwards', 'Neighborhood_StoneBr', 'Condition1_Artery', 'Condition1_RRAe', 'OverallCond_3', 'OverallCond_4', 'OverallCond_5', 'OverallCond_6', 'BsmtExposure_Gd', 'Heating_Grav', 'KitchenQual_Ex', 'Functional_Maj2', 
      'Functional_Mod', 'GarageType_2Types', 'GarageQual_Ex', 'SaleCondition_Normal']]

### Loss Functions and Model Development

In [0]:
# Loss functions
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def rmsle(y, predictions):
    return sqrt(mean_squared_log_error(y, predictions))

def convertToKaggle(predictions):
    finalDf = pd.DataFrame()
    test = pd.read_csv('./test.csv')
    finalDf['Id'] = test['Id'] # getting ID column for kaggle
    finalDf['SalePrice'] = pd.DataFrame(data= predictions, columns = ['SalePrice']) # gets SalesPrice data and converts array to DF
    finalDf.to_csv('kaggleFile.csv', index = False) # csv to Kaggle


### Fine-Tune XGBoost using GridSearch

In [0]:
def two_scorer():
    return make_scorer(rmsle) 

scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(rmsle)}

xgb1 = xgb.XGBRegressor()

parameters = {'learning_rate':[0.01, .02, .03, .04, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
              'n_estimators':[1000, 2000, 3460, 4000],                                       
              'max_depth':[2, 3, 4, 5, 6, 8, 10, 12, 15], 
              'min_child_weight':[1, 2, 3, 5, 7, 9],        
              'gamma':[0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4], 
              'subsample':[0.5, 0.9],                                     
              'colsample_bytree':[0.3, 0.5, 0.7, 0.9, 1.0],                                 
              'objective':['reg:squarederror', 'reg:squaredlogerror'], 
              'nthread':[4],                                 
              'scale_pos_weight':[0.7, 0.8, 0.9, 1, 1.1, 1.2], 
              'seed':[27],                                    
              'reg_alpha':[1e-5, 1e-2, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0], 
              'eta':[0.05, 0.1, 0.2, 0.3, 0.5],
              'predictor': ['gpu_predictor'], 
              'tree_method': ['gpu_hist'],
              'n_jobs':['-1']
               }

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        verbose=1,
                       scoring=two_scorer)
xgb_grid.fit(df,
         y)

results = xgb_grid.cv_results_
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)


### **Note that above code ran for 15 hours on GPU before crashing.  Ultimately, decided that manually testing different parameters to optimize is more efficient.**  

### Train and predict on full Data using parameters from GridSearch

In [0]:
xgboost = xgb.XGBRegressor(learning_rate=0.01,n_estimators=2300,
                                         max_depth=4, min_child_weight=2,
                                         gamma=0, subsample=0.54,
                                         colsample_bytree=0.52,
                                         objective='reg:squarederror', nthread=-1,
                                         scale_pos_weight=0.8, seed=27,
                                         reg_alpha=0.006, eta = 0.05, verbose_eval=False)

In [113]:
xgboost = xgb.XGBRegressor(learning_rate=0.011,n_estimators=2800,
                                         max_depth=4, min_child_weight=0,
                                         gamma=0.4, subsample=0.54,
                                         colsample_bytree=0.47,
                                         objective='reg:squarederror', nthread=-1,
                                         scale_pos_weight=0.8, seed=27,
                                         reg_alpha=1.0, eta = 0.5, verbose_eval=False)
modelCondensed = xgboost.fit(condensedData, y_train)
predictions = modelCondensed.predict(condensedTest)
print(rmsle(y_val, predictions)) # results for condensed

0.1257879589798264


In [0]:
modelFinal = xgboost.fit(df, y)
finalPredictions = modelFinal.predict(dfTest) # predicts on test dataset

### Train and Predict on condensedData

In [0]:
modelCondensed = xgboost.fit(condensedData, y_train)
predictions = modelCondensed.predict(condensedTest)

### See Results

In [109]:
print(rmsle(y_val, predictions)) # results for condensed

0.12642915565619572


### Convert to Kaggle

In [0]:
convertToKaggle(finalPredictions)

Ultimately, the full dataset that is trained performed best.  It is a shame the GPU crashed when performing GridSearch, so more processing power ultimately would have led to a better performance.